<a href="https://colab.research.google.com/github/ZerocoderForstudents/Working-with-videos/blob/main/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install requests
!pip install moviepy

In [ ]:
# Import modules
import requests
import os
from moviepy.editor import VideoFileClip, CompositeVideoClip, ImageClip

# Code

In [ ]:
# Settings
PIXABAY_API_KEY = '47304130-4b574d966d6ab0671cc18abfa'  # Replace with your API key

search_query = 'Data Scientist'  # Your search query
per_page = 3  # Number of videos to download

logo_path = '/content/google.png'  # Path to your logo

# Functions
def download_pixabay_videos(query, per_page=3):
    url = 'https://pixabay.com/api/videos/'

    params = {
        'key': PIXABAY_API_KEY,
        'q': query,
        'per_page': per_page,
        'safesearch': 'true'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        videos = response.json()['hits']
        video_paths = []
        for idx, video in enumerate(videos):
            # Selecting the video with the best quality
            video_files = video['videos']
            if 'large' in video_files:
                video_url = video_files['large']['url']
            elif 'medium' in video_files:
                video_url = video_files['medium']['url']
            else:
                video_url = video_files['small']['url']

            video_response = requests.get(video_url)
            video_filename = f'video_{idx}.mp4'
            with open(video_filename, 'wb') as f:
                f.write(video_response.content)
            video_paths.append(video_filename)
        return video_paths
    else:
        print('Error requesting Pixabay API:', response.status_code)
        return []

def overlay_logo_on_video(video_path, logo_path, output_path):
    # Load video
    video_clip = VideoFileClip(video_path)

    # Check for audio (to avoid warnings)
    if video_clip.audio is None:
        video_clip = video_clip.set_audio(None)

    # Load logo
    logo = (ImageClip(logo_path)
            .set_duration(video_clip.duration)
            .resize(height=100)  # Resize the logo if necessary
            .margin(right=8, top=8, opacity=0)  # Margins
            .set_pos(("right", "top")))  # Logo position

    # Overlay logo on video
    final_video = CompositeVideoClip([video_clip, logo])

    # Save the final video
    final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # Free up resources
    video_clip.close()
    final_video.close()

# Main process

video_paths = download_pixabay_videos(search_query, per_page)

if video_paths:
    for video_path in video_paths:
        output_path = f'logo_{video_path}'
        overlay_logo_on_video(video_path, logo_path, output_path)
        print(f'Video with overlaid logo saved: {output_path}')
else:
    print('Failed to download video.')

# Merging

In [ ]:
# Import modules
from moviepy.editor import VideoFileClip, clips_array

# Settings
video_paths = ['/content/logo_video_0.mp4', '/content/logo_video_1.mp4', '/content/logo_video_2.mp4']  # Replace with your files
output_path = 'merged_video.mp4'

# Load video files
video_clips = [VideoFileClip(video) for video in video_paths]

target_size = (640, 360)  # Set the desired size
resized_clips = [clip.resize(target_size) for clip in video_clips]

# Combine videos into one frame (in a row)
final_clip = clips_array([resized_clips])

# Save the final video
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Free up resources
for clip in resized_clips:
    clip.close()
final_clip.close()

print(f'Final video saved as {output_path}')